# CityScan Accessibility Analysis

- This analysis will measure accessibility to only schools
- Origins are derived from 100m grid blocks coming from WorldPop
- The accessibility measure is the time to the nearest facility in seconds

## Inputs:

#### origins: derived from a WorldPop raster manually using the 'Raster pixels to Points' tool in QGIS 3, and saved as a shapefile
#### destinations: The combination of POIs from Overpass as points and centroids of polygons (QGIS 3 Centoids tool). The two different point layers were merged with the QGIS 'Merge vector Layers' tool.
#### The admin AOI will be used to create a bounding box to download the road network via OSMNX


### Import libraries

In [6]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

import matplotlib.pyplot as plt

In [7]:
# Get reference to GOSTNets
sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

In [8]:
# define output folder
inputFolder = r'cityscan_accessibility_analysis_input_folder'
# define output folder
outputFolder = r'../../../cityscan_accessibility_analysis_output_folder'
# define road graph
roadGraph = os.path.join(inputFolder, "OSM_Roads.pickle")

In [9]:
# import extent
city_extent = gpd.read_file(os.path.join(inputFolder, "Addis_AOI.shp"))

In [10]:
# This is the section where we can adjust the road speeds
if not os.path.exists(roadGraph):
    extent = box(*city_extent.total_bounds)
    G = ox.graph_from_polygon(extent, network_type = 'drive_service')
    # This is how time is calculated from the OSMNX length attribute
    G = gn.convert_network_to_time(G, 'length')
    
    # compatible with NetworkX 2.4
    list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.strongly_connected_components(G))
    max_graph = None
    max_edges = 0
    for i in list_of_subgraphs:
        if i.number_of_edges() > max_edges:
            max_edges = i.number_of_edges()
            max_graph = i

    # set your graph equal to the largest sub-graph
    G = max_graph
    
    print('print G')
    print(sz)
    nx.write_gpickle(G, roadGraph)
else:
    G = nx.read_gpickle(roadGraph)    
    G = gn.convert_network_to_time(G, 'length')

print G


NameError: name 'sz' is not defined

### Inspect the road graph as a shapefile

In [ ]:
roads = gn.edge_gdf_from_graph(G)

In [ ]:
roads[:2]

In [ ]:
if not os.path.exists(outputFolder + '/OSM_Roads/OSM_Roads.shp'):
    roads['oneway'] = roads.oneway.astype(int)
    roads = roads[['oneway','length','time','mode','geometry']]
    roads.to_file(outputFolder + '/OSM_Roads')

## Create origins and destinations and snap them to the road graph

In [ ]:
# insert origins
origins = gpd.read_file(os.path.join(inputFolder, "world_pop_population_pts.shp"))

In [ ]:
# insert destinations
destinations = gpd.read_file(os.path.join(inputFolder, "merged_school_pts.shp"))

In [ ]:
# snap the origins to the road graph
snapped_origins = gn.pandana_snap_c(G, origins)

In [ ]:
# snap the destinations to the road graph
snapped_destinations = gn.pandana_snap_c(G, destinations)

In [ ]:
originNodes = list(snapped_origins['NN'].unique())

In [ ]:
destinationNodes = list(snapped_destinations['NN'].unique())

In [ ]:
# calculate OD matrix
OD_matrix = gn.calculate_OD(G, originNodes, destinationNodes, fail_value=-1, weight='time')

In [ ]:
#OD_matrix[:3]

In [ ]:
closest_facility_per_origin = OD_matrix.min(axis=1)

In [ ]:
closest_facility_per_origin[:5]

In [ ]:
# calculate accessibility
# For each row, the closest facility is the smallest value in the row

output = snapped_origins.copy()

closest_facility_per_origin = OD_matrix.min(axis=1)

results = pd.DataFrame([originNodes, closest_facility_per_origin]).transpose()
colName = "travel_time_to_closest_facility"
results.columns = ['NN', colName]

In [ ]:
results[:5]

In [ ]:
output = pd.merge(output, results, on = "NN")

In [ ]:
output.to_csv(os.path.join(outputFolder, "addis_accessibility_schools.csv"))